In [ ]:
%cd ~
import matplotlib
matplotlib.use("PDF")
import pyfits,glob,time,scipy, sys
from scipy.special import erf
from pylab import *
from subprocess import call

topdir='/Users/cslage/Research/LSST/code/GUI/'
#thedir=topdir+'brighter_fatter/20170831_002_spots/'
thedir=topdir+'brighter_fatter/20171128_002_spots_VBB60/'

%cd $thedir

configfile=topdir+'sextractor/default-array_dither.sex'
paramfile=topdir+'sextractor/default-array_dither.param'
maskdir=topdir+'sextractor/masks/'



In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

def make_reg_from_ldac(cat_ldac_file,text_tag):
    thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
    f = open(cat_ldac_file+'.reg','w')
    for i in range(len(thecat)):
        xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
        r=thecat['A_IMAGE'][i]
        thetext=thecat[text_tag][i]
        f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
    f.close()
    
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0



In [ ]:
# This definition runs sextractor in parallel and can be given to IPython's parallel map function along with
#   the file list
def ovsub_runsex_makereg(fitsfilename):
    import pyfits
    import numpy as np
    from subprocess import call
    topdir='/Users/cslage/Research/LSST/code/GUI/'
    thedir=topdir+'profiles'
    %cd $thedir

    configfile=topdir+'sextractor/default-array_dither.sex'
    paramfile=topdir+'sextractor/default-array_dither.param'
    maskdir=topdir+'sextractor/masks/'

    def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
        overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
        image=image[:y_overscan_start,:x_overscan_start]
        finalimg=image-np.matrix(np.median(overscan,axis=1)).T*np.ones(np.shape(image)[1])
        return finalimg

    def make_reg_from_ldac(cat_ldac_file,text_tag):
        thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
        f = open(cat_ldac_file+'.reg','w')
        for i in range(len(thecat)):
            xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
            r=thecat['A_IMAGE'][i]
            thetext=thecat[text_tag][i]
            f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
        f.close()
    for i in range(1,17):
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME']
        img=pyfits.getdata(fitsfilename,extname)
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022)   # cut off the overscan
        outname=fitsfilename[:-5]+extname+'.fits'
        pyfits.writeto(outname,overscansubimg,clobber=True)
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat'])
        make_reg_from_ldac(outname+'.cat','NUMBER')

In [ ]:
zfilelist=sort(glob.glob(thedir+'ITL-3800C-002_spot_spot_450_20171128??????.fits'))
print len(zfilelist)

In [ ]:
for fitsfilename in zfilelist: 
    tfile1=time.time() 
    for i in [13]:
        tstart=time.time() 
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME'] 
        img=pyfits.getdata(fitsfilename,extname) 
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
        # cut off the overscan 
        outname=fitsfilename[:-5]+extname+'.fits' 
        pyfits.writeto(outname,overscansubimg,clobber=True) 
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat']) 
        make_reg_from_ldac(outname+'.cat','NUMBER') 
        tend=time.time() 
        print extname+" time: "+str(tend-tstart)[:4] 
        print "Time taken for file "+str(fitsfilename[-26:-23])+": "+str(time.time()-tfile1)

#### Using the sextractor catalogs produced above to make a map of the sextractor measurement named below

In [ ]:
class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.imax=zeros([nstamps])


def BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 50
    xcoomax = 250
    ycoomin = 1500
    ycoomax = 1700
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    hdr=pyfits.getheader(fitsfilename,segmentnumber)
    extname = hdr['EXTNAME']
    img=pyfits.getdata(fitsfilename,extname) 
    overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
    catname=fitsfilename[:-5]+extname+'.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = overscansubimg[ymin:ymax, xmin:xmax]
           
            xsum = 0.0
            ysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    ysum += spotlist.y[j] * spotlist.data[i,j,n]
                    xsum += spotlist.x[i] * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.xoffset[n] = xoff
            spotlist.yoffset[n] = yoff
            #spotlist.xoffset[n] = xcoord - xint - 1.0
            #spotlist.yoffset[n] = ycoord - yint - 1.0     
            #print "Spot = %d, calc_off = (%f,%f), sex_off = (%f,%f)"%(n,xoff,yoff,xcoord-xint-1.0,ycoord-yint-1.0)
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist

def FOM(params):
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result

def PyFOM(params):
    fom = 0.0
    [Imax, sigmax, sigmay] = params
    area=zeros([spotlist.nx,spotlist.ny])
   
    for spot in range(spotlist.nstamps):
        for ii in range(spotlist.nx):
            for jj in range(spotlist.ny):
                xl = spotlist.x[ii] - spotlist.xoffset[spot] - 0.5
                xh = xl + 1.0
                yl = spotlist.y[jj] - spotlist.yoffset[spot] - 0.5
                yh = yl + 1.0
                #if spot == 78 and ii == 4 and jj == 4 or spot==0:
                    #print "ii = %d, jj = %d, img = %.4f"%(ii,jj,spotlist.data[ii,jj,spot])
                #print "ii = %d, jj = %d,xl = %.2f, xh = %.2f, yl = %.2f, yh = %.2f"%(ii,jj,xl,xh,yl,yh)
                area[ii,jj] = AreaG(xl, xh, yl, yh, sigmax, sigmay, Imax)
                fom += square(area[ii,jj]-spotlist.data[ii,jj,spot])
    #print "Imax = %.1f, sigmax = %.2f, sigmay = %.2f, fom = %.1f"%(Imax, sigmax, sigmay, fom)
    return fom


In [ ]:
nx = 11
ny = 11
numspots =100
segmentnumber = 13

fitsfilename = zfilelist[0]
print fitsfilename
    
spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, 0.7, 1.4)

print "nstamps = %d"%spotlist.nstamps
print "xoff = %f, yoff = %f"%(spotlist.xoffset[spot],spotlist.yoffset[spot])
starttime = time.time()
param0 = [1.00, 1.00]

args = ()#(spotlist)
Result = fmin_powell(FOM, param0, args)
elapsed = time.time() - starttime
print Result, "Elapsed Time = %f"%elapsed
testn = 0
spot = 23
sigmax = abs(Result[0])
sigmay = abs(Result[1])
Imax = spotlist.imax[spot] 
figure(figsize=(12,5))
suptitle=("Spot # %d"%(spot))
area=zeros([nx,ny])
for ii in range(nx):
    for jj in range(ny):
        xl = ii - int(nx/2) - spotlist.xoffset[spot] - 0.5
        xh = xl + 1.0
        yl = jj - int(ny/2) - spotlist.yoffset[spot] - 0.5
        yh = yl + 1.0
        area[ii,jj] = AreaG(xl, xh, yl, yh, sigmax, sigmay, Imax) 

subplots_adjust(wspace = 2.0)
subplot(1,3,1)
imshow(spotlist.data[:,:,spot] ,interpolation="None")
subplot(1,3,2)
plot(spotlist.data[int(nx/2),:,spot] , lw = 2, label="Data", color='red')
plot(area[int(nx/2),:], lw = 2, ls = '--', label="Model", color = 'green')
xlabel("X (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8])
ylim(0,25000)
subplot(1,3,3)
plot(spotlist.data[:,int(ny/2),spot], lw = 2, label="Data", color = 'red')
plot(area[:,int(ny/2)], lw = 2, ls = '--', label="Model", color = 'green')
xlabel("Y (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8,10])
legend(loc = (-6.5,0.8))
savefig("Typical_Fit_20171128_30Nov17_%s.pdf"%(testn))
show()
close()


In [ ]:
def CGIntegrand(y, xl, yl, c):
    # Convolved Gaussian Integrand.
    s2 = sqrt(2.0)
    yc = sqrt(c*c - y*y)
    yp = yc + xl
    ym = yc - xl
    f = yp * erf(yp / s2) - ym * erf(ym / s2)
    f += s2 * (exp(-yp * yp / 2.0) - exp(-ym * ym / 2.0)) / sqrt(pi)
    f *= erf((y+yl) / s2) - 1.0
    f -= 2.0 * yc * erf((y+yl) / s2)
    f += pi * c / 2.0
    return f / (4.0 * pi * c * c)


def Area(xl, xh, yl, yh, sigmax, sigmay, c):
    # Calculates how much of a 2D Convolved Gaussian falls within a rectangular box
    f = quad(CGIntegrand, -c, c, args=(xl/sigmax,yl/sigmay,c))[0]
    f += quad(CGIntegrand, -c, c, args=(xh/sigmax,yh/sigmay,c))[0]
    f -= quad(CGIntegrand, -c, c, args=(xl/sigmax,yh/sigmay,c))[0]
    f -= quad(CGIntegrand, -c, c, args=(xh/sigmax,yl/sigmay,c))[0]
    return f

def CGFOM(params):
    [sigmax, sigmay] = params
    #[sigmax, sigmay, c] = params
    if sigmax < 0.0 or sigmay < 0.0: return 1E12
    #forward_convolved.initialize(c)
    result = forward_convolved.evaluate(spotlist,sigmax,sigmay)
    #print sigmax, sigmay, c, result
    #sys.stdout.flush()
    return result


nx = 11
ny = 11
numspots = 100
segmentnumber = 13

fitsfilename = zfilelist[0]
print fitsfilename
sys.stdout.flush();
    
spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, 0.7, 1.4)

#c = 2.62
c = 3.1056
forward_convolved.initialize(c)

print "nstamps = %d"%spotlist.nstamps
starttime = time.time()
#param0 = [1.00, 1.00, 2.62]
param0 = [0.5, 0.5]

args = ()#(spotlist)
Result = fmin_powell(CGFOM, param0, args)

elapsed = time.time() - starttime
print Result, "Elapsed Time = %f"%elapsed


In [ ]:
close()
testn = 0
spot = 23
sigmax = abs(Result[0])
sigmay = abs(Result[1])
#c = Result[2]

Total = spotlist.data[:,:,spot].sum()
Imax = abs(spotlist.imax[spot] )
print sigmax, sigmay, c, Imax, Total
figure(figsize=(12,5))
suptitle=("Spot # %d"%(spot))
area=zeros([nx,ny])
for ii in range(nx):
    for jj in range(ny):
        xl = ii - int(nx/2) - spotlist.xoffset[spot] - 0.5
        xh = xl + 1.0
        yl = jj - int(ny/2) - spotlist.yoffset[spot] - 0.5
        yh = yl + 1.0
        area[ii,jj] = Imax * Area(xl, xh, yl, yh, sigmax, sigmay, c)

subplots_adjust(wspace = 2.0)
subplot(1,3,1)
imshow(spotlist.data[:,:,spot] ,interpolation="None")
subplot(1,3,2)
plot(spotlist.data[int(nx/2),:,spot] , lw = 2, label="Data", color='red')
plot(area[int(nx/2),:], lw = 2, ls = '--', label="Model", color = 'green')
xlabel("X (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8])
ylim(0,25000)
subplot(1,3,3)
plot(spotlist.data[:,int(ny/2),spot], lw = 2, label="Data", color = 'red')
plot(area[:,int(ny/2)], lw = 2, ls = '--', label="Model", color = 'green')
xlabel("Y (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8])
ylim(0,25000)
legend(loc = (-6.5,0.8))
savefig("Typical_Fit_Convolved_C_310_30Nov17_%s.pdf"%(testn))
show()
close()


In [ ]:
c * sigmax 